<a href="https://colab.research.google.com/github/SyrnikRebirth/qr_code_detector/blob/main/CV6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подтянем картинки

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [20]:
!git clone https://github.com/SyrnikRebirth/qr_code_detector.git

fatal: repository 'chto-to-tam' does not exist


А теперь зафиксируем в какой папке мы работаем

In [12]:
dataset_path = './drive/MyDrive/TestSet1'

Основная идея - для того чтобы найти find pattern надо найти вложенные друг в друга квадратики.

In [13]:
import cv2
import numpy as np

In [14]:
def img_preprocess(input_img):
    img = input_img.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Уменьшили шум
    img = cv2.GaussianBlur(img, (9, 9), 0)
    # Адаптивная бинаризация чтобы получить хорошее изображение для накидывания поиска контуров
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 13, 2)
    # Еще раз уменьшаем 
    img = cv2.medianBlur(img, 9)

    return img

In [15]:
def detect_find_patterns(hierarchy, contours):
    good_contours_indxs = []
    good_families = []
    # итерируемся по кандидатам
    for i in range(hierarchy.shape[0]):
        family = [i]
        child = hierarchy[i, 2]
        while child != -1:
            family.append(child)
            child = hierarchy[child, 2]
        if len(family) < 3:
            continue
        
        # Попытаемся найти форму контура, если в форме не 4 точки, то пропускаем
        approx = cv2.approxPolyDP(contours[i], 0.02*cv2.arcLength(contours[i], True), True)
        if len(approx) != 4:
            continue
        # Проверяем, что наш контур - это квадрат
        w = np.sqrt(np.sum((approx[0, 0] - approx[1, 0]) ** 2).astype(float))
        h = np.sqrt(np.sum((approx[1, 0] - approx[2, 0]) ** 2).astype(float))
        if abs(w - h) / min(w, h) > 0.25:
            continue
        # Проверяем, что внутри нашего контура есть другие контуры, площади которых относятся друг к другу как 49/25/9
        area_a = cv2.contourArea(contours[family[0]])
        area_b = cv2.contourArea(contours[family[1]])
        area_c = cv2.contourArea(contours[family[2]])
        if area_a == 0 or area_b == 0 or area_c == 0:
            continue
        ratio1 = area_a / area_b
        ratio2 = area_b / area_c
        if not (49 / 50) < ratio1 < (98 / 25) and not (25 / 18) < ratio2 < (50 / 9):
            continue
        good_contours_indxs.append(i)
        good_families.append(family)

    # У нас скопились родственные контуры их надо удалить, оставив только контуры-родители
    dups = set()
    for i in range(len(good_contours_indxs)):
        for j in range(len(good_families)):
            if i == j:
                continue
            if good_families[i][0] in good_families[j][1:]:
                dups.add(good_contours_indxs[i])
    final_contours_indxs = [candidate for candidate in good_contours_indxs if candidate not in dups]
    return final_contours_indxs

In [16]:
def draw_results(img, hierarchy, contours, contours_indxs, output_path):
    result_img = img.copy()
    color = (0, 255, 0)
    for indx in contours_indxs:
        cv2.drawContours(result_img, contours, indx, color, 5, cv2.LINE_8, hierarchy, 0)
    cv2.imwrite(output_path, result_img)

In [20]:
import os
import time

time_start = time.time()
for img_name in os.listdir(dataset_path):
    img = cv2.imread(dataset_path + '/' + img_name, cv2.IMREAD_COLOR)
    # Предобработаем изображение
    preprocessed_img = img_preprocess(img)
    # Найдем грани
    edges = cv2.Canny(preprocessed_img, 80, 120, 7)
    # Используем грани для поиска контуров
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy[0]
    # найдем среди конутров find pattern'ы
    final_contours_indxs = detect_find_patterns(hierarchy, contours)
    # Нарисуем наши результаты
    draw_results(img, hierarchy, contours, final_contours_indxs,'result_' + img_name)

time_spent = time.time() - time_start
print(f'Время работы: {time_spent} секунд')
print(f'{(time_spent / len(os.listdir(dataset_path)))} секунд/картинка')

Время работы: 32.11927890777588 секунд
0.6833889129314017 секунд/картинка
